In [ ]:
import requests
import openpyxl
import webbrowser
import time
chrome_path = 'C:/Program Files (x86)/Google/Chrome/Application/chrome.exe %s'

with open('YoutubeAPI.txt', 'r') as f:
    your_api_key = f.read()

def testa_site(url):
    try:
        url = requests.get(url, verify = False).url
        if str(requests.get(url, verify = False).status_code).startswith('2'):
            if requests.get(url, verify = False).headers.get('Set-Cookie'):
                if 'youtube.com' in requests.get(url, verify = False).headers.get('Set-Cookie'):
                    if url.find('&') == -1 and url.find('=') == -1:
                        id_of_video = url[url.rfind('/')+1:]
                    if url.find('&') == -1 and url.find('=') != -1:
                        id_of_video = url[url.find('=')+1:]
                    if url.find('&') != -1 and url.find('=') != -1:
                        id_of_video = url[url.find('=')+1: url.find('&')]
                    url_youtube = f'https://www.googleapis.com/youtube/v3/videos?id={id_of_video}&key={your_api_key}&part=status'
                    try:
                        if requests.get(url_youtube).json()['items'][0]['status']['publicStatsViewable'] == True:
                            return (True, "Youtube - OK")
                        else:
                            return (False, "Youtube - Erro")
                    except:
                        return (False, "Youtube - Erro")
                else:
                    return (True, 'Site - OK')
            else:
                return(True, 'Site - OK - Sem cookie')
        elif str(requests.get(url, verify = False).status_code).startswith('3'):
            return (False, "Site - Alerta")
        else:
            return (False, "Site - Erro")
    except Exception as e:
        print(e)
        return (False, "Erro")

In [ ]:
# Varre o excel desejado verificando o site com o auxílio da função

wb = openpyxl.load_workbook('Sites ensino medio.xlsx')
wsheet = wb['Planilha1']
wsheet.cell(2, 10).value = "Status"

erros = []

for i in range(3,700):
    if i%10 ==0:
        print(i)
    link_web = wsheet.cell(i,9).value
    link_original = wsheet.cell(i,8).value
    
    # Caso sem link web nem original
    if ((not link_web) and (not link_original)):
        continue
    
    #Caso só com web
    if ((link_web) and (not link_original)):
        testado_web = testa_site(link_web)
        if not testado_web[0]:
            erros.append('linha ' + str(i) + ' - ' + link_web)
            wsheet.cell(i, 10).value = testado_web[1]
    
    if link_original:
        link_original = link_original.strip('_\t ')
        if (link_original.startswith('http') or link_original.startswith('www') or link_original.startswith('youtu') or link_original.startswith('bit.')):
            
            #Caso só com original
            if (not link_web):
                testado_original = testa_site(link_original)
                if not testado_original[0]:
                    erros.append('linha ' + str(i) + ' - ' + link_original)
                    wsheet.cell(i, 10).value = testado_original[1]
            
            # Caso com os dois
            if (link_web and link_original):
                
                # Teste de redirecionamento
                try:
                    redir = requests.get(link_web, verify = False).url
                    if link_original != redir:
                        wsheet.cell(i, 10).value = "Erro redirect"
                        erros.append([link_web, link_original])
                except:
                    print(f'{i} - Erro de redirecionamento')
                    erros.append('linha ' + str(i) + ' - ' + link_original)
                
                testado_web = testa_site(link_web)
                testado_original = testa_site(link_original)
                if not testado_web[0]:
                    erros.append('linha ' + str(i) + ' - ' + link_web)
                    wsheet.cell(i, 10).value = testado_web[1]
                
                elif not testado_original[0]:
                    erros.append('linha ' + str(i) + ' - ' + link_original)
            

wb.save('Novo Medio.xlsx')
print(erros)

In [ ]:
#Abre os sites unitários que apresentaram chance de erro

i = 1
for site in erros:
    if not isinstance(site, list):
        print(i, site)
        i+=1
for site in reversed(erros):
    if not isinstance(site, list):
        link = site[site.find('-')+2:]
        webbrowser.get(chrome_path).open_new_tab(link)
        time.sleep(.4)

In [ ]:
#Abre os pares de sites que deram erros de redirecionamento

for site in reversed(erros):
    if isinstance(site, list):
        print(site)
        webbrowser.get(chrome_path).open_new_tab(site[0])
        webbrowser.get(chrome_path).open_new_tab(site[1])
        time.sleep(.4)